# Görüntü Sınıflandırma için Derin Sinir Ağı: Uygulama

Derin bir ağ oluşturmak için önceki çalışma dosyalarında uyguladığınız fonksiyonları kullanacak ve bunu kedi ve kedi olmayan sınıflandırmaya uygulayacaksınız. Umarım, önceki lojistik regresyon uygulamanıza göre doğrulukta bir gelişme görürsünüz.

Başlayalım!

## 1 - Paketler

Öncelikle bu ödev sırasında ihtiyaç duyacağınız tüm paketleri import edelim.
- [numpy](www.numpy.org), Python ile bilimsel hesaplama için temel pakettir.
- [matplotlib](http://matplotlib.org), Python'da grafikler çizmek için bir kütüphanedir.
- [h5py](http://www.h5py.org), bir H5 dosyasında depolanan bir veri kümesiyle etkileşim kurmak için yaygın bir pakettir.
- [PIL](http://www.pythonware.com/products/pil/) ve [scipy](https://www.scipy.org/) burada sonunda kendi resminizle modelinizi test etmek için kullanılır .
- np.random.seed(1), tüm rastgele işlev çağrılarını tutarlı tutmak için kullanılır. Çalışmanızı derecelendirmemize yardımcı olacaktır.

In [ ]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
#from dnn_app_utils import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)


In [ ]:
#@title Yardımcı Fonksiyonlar
#helper functions

def sigmoid(Z):
    """
    Implements the sigmoid activation in numpy
    
    Arguments:
    Z -- numpy array of any shape
    
    Returns:
    A -- output of sigmoid(z), same shape as Z
    cache -- returns Z as well, useful during backpropagation
    """
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

def relu(Z):
    """
    Implement the RELU function.

    Arguments:
    Z -- Output of the linear layer, of any shape

    Returns:
    A -- Post-activation parameter, of the same shape as Z
    cache -- a python dictionary containing "A" ; stored for computing the backward pass efficiently
    """
    
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache


def relu_backward(dA, cache):
    """
    Implement the backward propagation for a single RELU unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ


def load_data():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(1)
    
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters     


def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1]) #*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python dictionary containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    Z = W.dot(A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = 1./m * np.dot(dZ,A_prev.T)
    db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T,dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (there are (L-1) or them, indexes from 0 to L-2)
                the cache of linear_activation_forward() with "sigmoid" (there is one, index L-1)
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1]
    grads["dA" + str(L)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 2)], current_cache, activation = "relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        
    return parameters

def predict(X, y, parameters):
    """
    This function is used to predict the results of a  L-layer neural network.
    
    Arguments:
    X -- data set of examples you would like to label
    parameters -- parameters of the trained model
    
    Returns:
    p -- predictions for the given dataset X
    """
    
    m = X.shape[1]
    n = len(parameters) // 2 # number of layers in the neural network
    p = np.zeros((1,m))
    
    # Forward propagation
    probas, caches = L_model_forward(X, parameters)

    
    # convert probas to 0/1 predictions
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    
    #print results
    #print ("predictions: " + str(p))
    #print ("true labels: " + str(y))
    print("Accuracy: "  + str(np.sum((p == y)/m)))
        
    return p

def print_mislabeled_images(classes, X, y, p):
    """
    Plots images where predictions and truth were different.
    X -- dataset
    y -- true labels
    p -- predictions
    """
    a = p + y
    mislabeled_indices = np.asarray(np.where(a == 1))
    plt.rcParams['figure.figsize'] = (40.0, 40.0) # set default size of plots
    num_images = len(mislabeled_indices[0])
    for i in range(num_images):
        index = mislabeled_indices[1][i]
        
        plt.subplot(2, num_images, i + 1)
        plt.imshow(X[:,index].reshape(64,64,3), interpolation='nearest')
        plt.axis('off')
        plt.title("Prediction: " + classes[int(p[0,index])].decode("utf-8") + " \n Class: " + classes[y[0,index]].decode("utf-8"))


## 2 - Veri Kümesi

"Kedi vs Kedi olmayan" veri kümesini kullanacaksınız. 

**Problem Açıklaması**: Size aşağıdakileri içeren bir veri kümesi ("data.h5") verilir:
- kedi (y=1) veya kedi olmayan (y=0) olarak etiketlenen m_train görüntülerinden oluşan bir eğitim seti
- kedi veya kedi olmayan olarak etiketlenmiş bir m_test görüntüleri test seti
- her görüntü şekle (num_px, num_px, 3) sahiptir, burada 3, 3 kanal (RGB) içindir. Böylece, her görüntü karedir (yükseklik = num_px) ve (genişlik = num_px).

Resimleri kedi veya kedi olmayan olarak doğru bir şekilde sınıflandırabilen basit bir görüntü tanıma algoritması oluşturacaksınız.

Veri setini daha yakından tanıyalım. Aşağıdaki kodu çalıştırarak verileri yükleyin.

In [ ]:
ROOT_DIR = "https://raw.githubusercontent.com/yapay-ogrenme/casgem-eu-project-training-on-data-mining/main/PART2/Day9-NeuralNetworks/notebooks/"

DATASET_PATH = ROOT_DIR + "datasets/"

In [ ]:
DATASET_PATH_TRAIN = DATASET_PATH + 'train_catvnoncat.h5'
DATASET_PATH_TEST = DATASET_PATH + 'test_catvnoncat.h5'

!wget $DATASET_PATH_TRAIN
!wget $DATASET_PATH_TEST

In [ ]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

Aşağıdaki kod size veri setinde bir resim gösterecektir. Diğer görüntüleri görmek için index değerini değiştirmekten ve hücreyi birden çok kez yeniden çalıştırmaktan çekinmeyin.

In [ ]:
# Example of a picture
index = 10
plt.imshow(train_x_orig[index])
print ("y = " + str(train_y[0, index]) + ". It's a " + classes[train_y[0, index]].decode("utf-8") + " picture.")

In [ ]:
# Explore your dataset 
m_train = train_x_orig.shape[0]
num_px = train_x_orig.shape[1]
m_test = test_x_orig.shape[0]

print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_x_orig shape: " + str(train_x_orig.shape))
print ("train_y shape: " + str(train_y.shape))
print ("test_x_orig shape: " + str(test_x_orig.shape))
print ("test_y shape: " + str(test_y.shape))

Her zamanki gibi, görüntüleri ağa beslemeden önce yeniden şekillendirir ve standart hale getirirsiniz. Kod aşağıdaki hücrede verilmiştir.

<img src="https://drive.google.com/uc?id=1PvwKiRhk0wrIvNOInbIg4VMGKH-YQJVr" style="width:450px;height:300px;" alt="imvectorkiank" title="imvectorkiank">

<caption><center> <u>Şekil 1</u>: Görüntüden vektöre dönüştürme. <br> </center></caption>

In [ ]:
# Reshape the training and test examples 
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten / 255.
test_x = test_x_flatten / 255.

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))


$12.288$, yeniden şekillendirilmiş bir görüntü vektörünün boyutu olan 64 $ \times 64 \times 3$'a eşittir.

## 3 - Modelinizin mimarisi

Artık veri kümesine aşina olduğunuza göre, kedi görüntülerini kedi olmayan görüntülerden ayırt etmek için derin bir sinir ağı oluşturmanın zamanı geldi.

İki farklı model oluşturacaksınız:
- 2 katmanlı bir sinir ağı
- L katmanlı derin sinir ağı

Daha sonra bu modellerin performansını karşılaştıracak ve ayrıca $L$ için farklı değerler deneyeceksiniz.

Şimdi iki mimariye bakalım.

### 3.1 - 2 katmanlı sinir ağı

<img src="https://drive.google.com/uc?id=1xmAkJs4aDmXGuUWz0l38EI11wsS6t9yu" style="width:650px;height:400px;" alt="2layerNN_kiank" title="2layerNN_kiank">

<caption><center> <u>Şekil 2</u>: 2 katmanlı sinir ağı. <br> Model şu şekilde özetlenebilir: ***INPUT -> LINEAR -> RELU -> LINEAR -> SIGMOID -> OUTPUT***. </center></caption>


<u>Şekil 2'nin Ayrıntılı Mimarisi</u>:
- Girdi, $(12288,1)$ boyutunda bir vektöre düzleştirilmiş bir (64,64,3) görüntüdür.
- Karşılık gelen vektör: $[x_0,x_1,...,x_{12287}]^T$, daha sonra $(n^{[1]}) boyutundaki $W^{[1]}$ ağırlık matrisi ile çarpılır. , 12288)$.
- Daha sonra bir önyargı terimi eklersiniz ve aşağıdaki vektörü elde etmek için relu değerini alırsınız: $[a_0^{[1]}, a_1^{[1]},..., a_{n^{[1]}-1 }^{[1]}]^T$.
- Daha sonra aynı işlemi tekrarlıyorsunuz.
- Elde edilen vektörü $W^{[2]}$ ile çarpar ve bias (önyargı) eklersiniz.
- Son olarak, sonucun sigmoidini alırsınız. 0,5'ten büyükse, onu kedi olarak sınıflandırırsınız.

### 3.2 - L katmanlı derin sinir ağı

Yukarıdaki temsil ile bir L-katmanlı derin sinir ağını temsil etmek zordur. Ancak, burada basitleştirilmiş bir ağ gösterimi verilmiştir:

<img src="https://drive.google.com/uc?id=1m8-xt3u_xpTNFLjn-VVZ5KrsUSLVegRV" style="width:650px;height:400px;" alt="LlayerNN_kiank" title="LlayerNN_kiank">


<caption><center> <u>Şekil 3</u>: L katmanlı derin sinir ağı. <br> Model şu şekilde özetlenebilir: ***[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID***</center></caption>

<u>Şekil 3'ün Ayrıntılı Mimarisi</u>:
- Girdi, (12288,1) boyutunda bir vektöre düzleştirilmiş bir (64,64,3) görüntüdür.
- Karşılık gelen vektör: $[x_0,x_1,...,x_{12287}]^T$ daha sonra $W^{[1]}$ ağırlık matrisi ile çarpılır ve sonra $b^{[ kesişimini eklersiniz 1]}$. Sonuç lineer birim olarak adlandırılır.
- Ardından, lineer birimin relu'sunu alırsınız. Bu işlem, model mimarisine bağlı olarak her bir $(W^{[l]}, b^{[l]})$ için birkaç kez tekrarlanabilir.
- Son olarak, son lineer birimin sigmoidini alırsınız. 0,5'ten büyükse, onu kedi olarak sınıflandırırsınız.

### 3.3 - Genel metodoloji

Her zamanki gibi, modeli oluşturmak için Derin Öğrenme metodolojisini izleyeceksiniz:
  1. Parametreleri başlat / Hiperparametreleri tanımla
  2. Num_iterations için döngü:
    a. ileri yayılma
    b. Hesaplama maliyeti işlevi
    c. Geriye yayılma
    d. Parametreleri güncelleyin (parametreleri ve backprop'tan gelen notları kullanarak)
  3. Etiketleri tahmin etmek için eğitilmiş parametreleri kullanın

Şimdi bu iki modeli uygulayalım!

## 4 - İki katmanlı sinir ağı

**Soru**: Aşağıdaki yapıya sahip 2 katmanlı bir sinir ağı oluşturmak için önceki atamada uyguladığınız yardımcı fonksiyonları kullanın: *LINEAR -> RELU -> LINEAR -> SIGMOID*. İhtiyaç duyabileceğiniz fonksiyonlar ve girdileri şunlardır:


```python
def initialize_parameters(n_x, n_h, n_y):
    ...
    return parameters 
def linear_activation_forward(A_prev, W, b, activation):
    ...
    return A, cache
def compute_cost(AL, Y):
    ...
    return cost
def linear_activation_backward(dA, cache, activation):
    ...
    return dA_prev, dW, db
def update_parameters(parameters, grads, learning_rate):
    ...
    return parameters
```

In [ ]:
### CONSTANTS DEFINING THE MODEL ####
n_x = 12288     # num_px * num_px * 3
n_h = 7
n_y = 1
layers_dims = (n_x, n_h, n_y)

In [ ]:
# GRADED FUNCTION: two_layer_model

def two_layer_model(X, Y, layers_dims, learning_rate=0.0075, num_iterations=3000, print_cost=False):
    """
    Implements a two-layer neural network: LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (n_x, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- dimensions of the layers (n_x, n_h, n_y)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- If set to True, this will print the cost every 100 iterations 
    
    Returns:
    parameters -- a dictionary containing W1, W2, b1, and b2
    """
    
    np.random.seed(1)
    grads = {}
    costs = []                              # to keep track of the cost
    m = X.shape[1]                           # number of examples
    (n_x, n_h, n_y) = layers_dims
    
    # Initialize parameters dictionary, by calling one of the functions you'd previously implemented
    ### START CODE HERE ### (≈ 1 line of code)
    parameters = initialize_parameters(n_x, n_h, n_y)
    ### END CODE HERE ###
    
    # Get W1, b1, W2 and b2 from the dictionary parameters.
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> SIGMOID. Inputs: "X, W1, b1". Output: "A1, cache1, A2, cache2".
        ### START CODE HERE ### (≈ 2 lines of code)
        A1, cache1 = linear_activation_forward(X, W1, b1, 'relu')
        A2, cache2 = linear_activation_forward(A1, W2, b2, 'sigmoid')
        ### END CODE HERE ###
        
        # Compute cost
        ### START CODE HERE ### (≈ 1 line of code)
        cost = compute_cost(A2, Y)
        ### END CODE HERE ###
        
        # Initializing backward propagation
        dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))
        
        # Backward propagation. Inputs: "dA2, cache2, cache1". Outputs: "dA1, dW2, db2; also dA0 (not used), dW1, db1".
        ### START CODE HERE ### (≈ 2 lines of code)
        dA1, dW2, db2 = linear_activation_backward(dA2, cache2, 'sigmoid')
        dA0, dW1, db1 = linear_activation_backward(dA1, cache1, 'relu')
        ### END CODE HERE ###
        
        # Set grads['dWl'] to dW1, grads['db1'] to db1, grads['dW2'] to dW2, grads['db2'] to db2
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2
        
        # Update parameters.
        ### START CODE HERE ### (approx. 1 line of code)
        parameters = update_parameters(parameters, grads, learning_rate)
        ### END CODE HERE ###

        # Retrieve W1, b1, W2, b2 from parameters
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if print_cost and i % 100 == 0:
            costs.append(cost)
       
    # plot the cost

    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

Parametrelerinizi eğitmek için aşağıdaki hücreyi çalıştırın. Modelinizin çalışıp çalışmadığına bakın. Maliyet azalıyor olmalı. 2500 iterasyonda çalıştırmak 5 dakika kadar sürebilir. "İlk iterasyondan sonra maliyet"in aşağıdaki beklenen çıktıyla eşleşip eşleşmediğini kontrol edin

In [ ]:
parameters = two_layer_model(train_x, train_y, layers_dims = (n_x, n_h, n_y), num_iterations = 2500, print_cost=True)

**Beklenen Çıktı**:
<table> 
    <tr>
        <td> Cost after iteration 0</td>
        <td> 0.693049735659989 </td>
    </tr>
    <tr>
        <td> Cost after iteration 100</td>
        <td> 0.6464283150388817 </td>
    </tr>
    <tr>
        <td>...</td>
        <td>...</td>
    </tr>
    <tr>
        <td> Cost after iteration 2400</td>
        <td> 0.04950829635846386 </td>
    </tr>
</table>

İyi ki vektörleştirilmiş bir uygulama oluşturmuşsunuz! Aksi takdirde, bunu eğitmek 10 kat daha uzun sürebilirdi.

Artık, veri kümesindeki görüntüleri sınıflandırmak için eğitilmiş parametreleri kullanabilirsiniz. Eğitim ve test setleri ile ilgili tahminlerinizi görmek için aşağıdaki hücreyi çalıştırın.

In [ ]:
predictions_train = predict(train_x, train_y, parameters)

In [ ]:
predictions_test = predict(test_x, test_y, parameters)

**Not**: Modeli daha az iterasyonda çalıştırmanın (1500 diyelim) test kümesinde daha iyi doğruluk sağladığını fark edebilirsiniz. Buna "erken durdurma(early stopping)" denir. Erken durdurma, aşırı uyumu önlemenin bir yoludur.

## 5 - L Katmanlı Sinir Ağı

**Soru**: Aşağıdaki yapıya sahip bir $L$-katmanlı sinir ağı oluşturmak için daha önce uyguladığınız yardımcı fonksiyonları kullanın: *[LINEAR -> RELU]$\times$(L-1) -> LINEAR -> SIGMOID*. İhtiyaç duyabileceğiniz fonksiyonlar ve girdileri şunlardır:


```python
def initialize_parameters_deep(layer_dims):
    ...
    return parameters 
def L_model_forward(X, parameters):
    ...
    return AL, caches
def compute_cost(AL, Y):
    ...
    return cost
def L_model_backward(AL, Y, caches):
    ...
    return grads
def update_parameters(parameters, grads, learning_rate):
    ...
    return parameters
```

In [ ]:
### CONSTANTS ###
layers_dims = [12288, 20, 7, 5, 1] #  5-layer model

In [ ]:
# GRADED FUNCTION: n_layer_model

def L_layer_model(X, Y, layers_dims, learning_rate=0.0075, num_iterations=3000, print_cost=False): #lr was 0.009
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (number of examples, num_px * num_px * 3)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization.
    ### START CODE HERE ###
    parameters = initialize_parameters_deep(layers_dims)
    ### END CODE HERE ###
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        ### START CODE HERE ### (≈ 1 line of code)
        AL, caches = L_model_forward(X, parameters)
        ### END CODE HERE ###
        
        # Compute cost.
        ### START CODE HERE ### (≈ 1 line of code)
        cost = compute_cost(AL, Y)
        ### END CODE HERE ###
    
        # Backward propagation.
        ### START CODE HERE ### (≈ 1 line of code)
        grads = L_model_backward(AL, Y, caches)
        ### END CODE HERE ###
 
        # Update parameters.
        ### START CODE HERE ### (≈ 1 line of code)
        parameters = update_parameters(parameters, grads, learning_rate)
        ### END CODE HERE ###
                
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per tens)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

Şimdi modeli 5 katmanlı bir sinir ağı olarak eğiteceksiniz.

Modelinizi eğitmek için aşağıdaki hücreyi çalıştırın. Maliyet her iterasyonda düşmelidir. 2500 iterasyonu çalıştırmak 5 dakika kadar sürebilir.

In [ ]:
parameters = L_layer_model(train_x, train_y, layers_dims, num_iterations=2500, print_cost=True)

In [ ]:
pred_train = predict(train_x, train_y, parameters)

In [ ]:
pred_test = predict(test_x, test_y, parameters)

Tebrikler! 5 katmanlı sinir ağınızın aynı test kümesinde 2 katmanlı sinir ağınızdan (%72) daha iyi performansa (%84) sahip olduğu görülüyor.

Bu, bu görev için iyi bir performans. İyi iş!

## 6) Sonuç Analizi

İlk olarak, L-layer modelinin yanlış etiketlediği bazı resimlere bakalım. Bu, birkaç yanlış etiketlenmiş resim gösterecektir.

In [ ]:
print_mislabeled_images(classes, test_x, test_y, pred_test)

**Modelin başarısız olma eğiliminde olduğu birkaç resim türü şunları içerir:**
- Alışılmadık bir pozisyonda kedi gövdesi
- Kedi benzer renkte bir arka planda belirir
- Olağandışı kedi rengi ve türleri
- Kamera açısı
- Resmin parlaklığı
- Ölçek varyasyonu (kedi görüntüde çok büyük veya küçüktür)